# Data analysis

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import urllib
import json
import pandas as pd
import os

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/home/marco/.wdm/drivers/chromedriver/linux64/95.0.4638.54/chromedriver] found in cache
/tmp/ipykernel_139976/2406386856.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
driver.get("https://sarc.asystom.com/monitor/login")

elem = driver.find_element_by_name("user")
elem.clear()
elem.send_keys(os.environ["SITE_USER"])

elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(os.environ["SITE_PASSWORD"])

btn = driver.find_element_by_class_name("css-mij01a-button")
btn.click()

/tmp/ipykernel_139976/4134849667.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("user")
/tmp/ipykernel_139976/4134849667.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("password")
/tmp/ipykernel_139976/4134849667.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_class_name("css-mij01a-button")


In [3]:
def get_dfs(parsed_json):
    dfs = []
    # parsed_json["results"][0]["series"][0]["columns"]
    # parsed_json["results"][0]["series"][0]["values"]

    for result in parsed_json["results"]:
        print("Result", result["statement_id"])
        for serie in result["series"]:
            df = pd.DataFrame(serie["values"], columns=serie["columns"])
            df.name = serie["name"]
            dfs.append(df)
    return dfs

In [4]:
from urllib import parse
q="SELECT%20median(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(2h)%20fill(none)%3BSELECT%20moving_average(%22decimated%22%2C%2050)%20FROM%20(SELECT%20median(%22drift%22)%20AS%20%22decimated%22%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(1m)%20fill(none))"

q="https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT%20last(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%201d%20AND%20%22mileage%22%252%3D1)%0A&epoch=ms"
q_clean = parse.unquote(q)
q_clean

'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT last("drift") FROM (SELECT 20*log10("vib_x_vel"*"vib_y_vel"*"vib_z_vel") AS "drift" FROM "Signature_Drift" WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - 1d AND "mileage"%2=1)\n&epoch=ms'

In [5]:
# SELECT vib_x_vel, vib_y_vel, vib_z_vel

delay = "200d"

q_clean = f"""
SELECT *
FROM "Signature_Drift"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Signature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Temperature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Current_Loop"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};
"""

q_econded = parse.quote(q_clean)
url = f'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q={q_econded}&epoch=ms'

driver.get(url)
driver.page_source
content = driver.find_element_by_tag_name('pre').text
parsed_json = json.loads(content)
parsed_json
dfs = get_dfs(parsed_json)

/tmp/ipykernel_139976/3831526946.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content = driver.find_element_by_tag_name('pre').text


Result 0
Result 1
Result 2
Result 3


In [6]:
with open("all_data.json", "w") as f:
    json.dump(parsed_json, f)

In [7]:
print(dfs[0].name)
dfs[0]

Signature_Drift


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1632822027177,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,258,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1
1,1632822331552,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.603113,0.428016,1.520943,0.162509,0.070954,...,1.057239,1.102804,1.0,1.0,1.0,0.930266,0.992188,1.0,1.152778,1
2,1632822626555,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,1.068894,0.929274,1.863890,1.574731,0.604257,...,1.106270,1.089286,1.0,1.0,1.0,1.433232,1.603636,1.0,1.103600,1
3,1632822926402,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.295262,-0.921836,-0.814832,0.993935,0.504692,...,1.054773,1.013333,1.0,1.0,1.0,1.080572,1.076412,1.0,1.035784,1
4,1632823226427,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.441291,-0.576333,1.248798,2.314946,0.782788,...,1.028589,1.066434,1.0,1.0,1.0,1.477110,1.229960,1.0,1.038660,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13748,1636299812175,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-3.407621,-1.539916,-0.853622,-0.498609,0.318753,...,1.126588,1.507937,1.0,1.0,1.0,0.938010,1.298492,1.0,3.220080,1
13749,1636299932138,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-3.254268,-2.004554,-1.860717,-0.656539,0.321042,...,1.099765,1.507937,1.0,1.0,1.0,0.976378,1.537186,1.0,2.670683,1
13750,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-1.910711,-1.997687,-1.006975,-0.219369,-0.239727,...,1.135529,1.545635,1.0,1.0,1.0,1.028378,1.384422,1.0,3.632129,1
13751,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-2.357037,-0.743395,-1.405235,-0.047705,0.135645,...,1.166824,1.583333,1.0,1.0,1.0,0.943563,1.346231,1.0,3.616867,1


In [8]:
print(dfs[1].name)
dfs[1]

Signature


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.608835,-71.288624,-71.528954,-72.222477,-72.794690,...,0.108339,0.006348,0.0,0.0,0.0,3.065537,0.024659,0.0,0.090028,0
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.000229,-71.844816,-71.519799,-72.007324,-72.817578,...,0.076295,0.006104,0.0,0.0,0.0,3.060960,0.023438,0.0,0.076295,0
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.306706,-71.922637,-72.487984,-72.142367,-72.199588,...,0.093080,0.006104,0.0,0.0,0.0,3.042649,0.022950,0.0,0.100710,0
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.593042,-71.657130,-72.716869,-72.259098,-73.073930,...,0.105287,0.006104,0.0,0.0,0.0,2.975509,0.021729,0.0,0.073243,0
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.158160,-71.458000,-72.158389,-71.586175,-72.877089,...,0.065614,0.006104,0.0,0.0,0.0,2.957198,0.022461,0.0,0.094606,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941,1636299812175,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-61.160449,-66.754406,-70.917830,-72.499428,-72.542916,...,0.384527,0.009766,0.0,0.0,0.0,2.835126,0.033204,0.0,0.321965,0
13942,1636299932138,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-61.007095,-67.219043,-71.924926,-72.657359,-72.540627,...,0.375372,0.009766,0.0,0.0,0.0,2.951095,0.039307,0.0,0.267033,0
13943,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-59.663539,-67.212177,-71.071183,-72.220188,-73.101396,...,0.387579,0.010010,0.0,0.0,0.0,3.108263,0.035401,0.0,0.363165,0
13944,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-60.109865,-65.957885,-71.469444,-72.048524,-72.726024,...,0.398260,0.010254,0.0,0.0,0.0,2.851911,0.034424,0.0,0.361639,0


In [9]:
print(dfs[2].name)
dfs[2]

Temperature


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
...,...,...,...,...,...,...
17582,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday
17583,1636300111833,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.253365,Sunday
17584,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday
17585,1636300231821,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday


In [10]:
print(dfs[3].name)
dfs[3]

Current_Loop


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
...,...,...,...,...,...,...
17582,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.949187,Sunday
17583,1636300111833,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.941405,Sunday
17584,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.943694,Sunday
17585,1636300231821,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.946899,Sunday
